In [1]:
import pandas as pd
import numpy as np
import os 
os.chdir("../")
os.chdir("FinalDataset/URL")

In [2]:
data = pd.read_csv("URL_feature.csv")
data.drop(columns='Unnamed: 0',inplace=True)

In [3]:
set(data["File"].tolist())

{'Benign_list_big_final',
 'DefacementSitesURLFiltered',
 'Malware_dataset',
 'phishing_dataset',
 'spam_dataset'}

In [4]:
data.replace('Benign_list_big_final',"Benign",inplace=True)
data.replace('DefacementSitesURLFiltered',"Defacement",inplace=True)
data.replace('Malware_dataset',"Malware",inplace=True)
data.replace('phishing_dataset',"Phishing",inplace=True)
data.replace('spam_dataset',"Spam",inplace=True)

In [5]:
data.head()

,File,bodyLength,bscr,dse,dsr,entropy,ext,hasHttp,hasHttps,numDigits,numImages,numLinks,numParams,numTitles,sbr,scriptLength,specialChars,sscr,urlIsLive,urlLength
0,Benign,0,0,0,0,-4.876201,to,True,False,18,0,0,0,0,0,0,0,0,False,83
1,Benign,0,0,0,0,-4.920700,to,True,False,23,0,0,0,0,0,0,0,0,False,83
2,Benign,0,0,0,0,-4.821630,to,True,False,22,0,0,0,0,0,0,0,0,False,83
3,Benign,0,0,0,0,-4.889040,to,True,False,18,0,0,0,0,0,0,0,0,False,83
4,Benign,0,0,0,0,-4.772234,to,True,False,18,0,0,0,0,0,0,0,0,False,83


In [6]:
grouped_df = data.groupby("File")
mean_df = grouped_df.mean()

In [7]:
mean_df.head()

,bodyLength,bscr,dse,dsr,entropy,hasHttp,hasHttps,numDigits,numImages,numLinks,numParams,numTitles,sbr,scriptLength,specialChars,sscr,urlIsLive,urlLength
File,,,,,,,,,,,,,,,,,,
Benign,0.0,0.0,0.0,0.0,-4.480162,0.945588,0.055119,15.083216,0.0,0.0,0.465120,0.0,0.0,0.0,0.0,0.0,0.0,115.007010
Defacement,0.0,0.0,0.0,0.0,-4.458097,1.000000,0.000000,4.592482,0.0,0.0,1.730180,0.0,0.0,0.0,0.0,0.0,0.0,86.339395
Malware,0.0,0.0,0.0,0.0,-4.269329,0.974754,0.007349,16.075307,0.0,0.0,0.723586,0.0,0.0,0.0,0.0,0.0,0.0,83.705948
Phishing,0.0,0.0,0.0,0.0,-4.474045,0.983241,0.032112,8.868741,0.0,0.0,0.222679,0.0,0.0,0.0,0.0,0.0,0.0,84.313999
Spam,0.0,0.0,0.0,0.0,-4.748068,1.000000,0.000000,21.736667,0.0,0.0,1.223167,0.0,0.0,0.0,0.0,0.0,0.0,126.761917


In [8]:
data.describe()

,bodyLength,bscr,dse,dsr,entropy,numDigits,numImages,numLinks,numParams,numTitles,sbr,scriptLength,specialChars,sscr,urlLength
count,165366.0,165366.0,165366.0,165366.0,165366.000000,165366.000000,165366.0,165366.0,165366.000000,165366.0,165366.0,165366.0,165366.0,165366.0,165366.000000
mean,0.0,0.0,0.0,0.0,-4.471618,9.141752,0.0,0.0,1.261499,0.0,0.0,0.0,0.0,0.0,95.099549
std,0.0,0.0,0.0,0.0,0.345106,15.712669,0.0,0.0,2.003503,0.0,0.0,0.0,0.0,0.0,61.457045
min,0.0,0.0,0.0,0.0,-5.978011,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,17.000000
25%,0.0,0.0,0.0,0.0,-4.662043,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,63.000000
50%,0.0,0.0,0.0,0.0,-4.486841,4.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,88.000000
75%,0.0,0.0,0.0,0.0,-4.249381,10.000000,0.0,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,110.000000
max,0.0,0.0,0.0,0.0,-2.129713,1204.000000,0.0,0.0,39.000000,0.0,0.0,0.0,0.0,0.0,2081.000000


In [9]:
data.drop(columns = ["bscr",'dse','dsr','numImages','numLinks','numTitles','sbr','scriptLength','specialChars','sscr','bodyLength','numParams'],inplace=True)

In [10]:
data.head()

,File,entropy,ext,hasHttp,hasHttps,numDigits,urlIsLive,urlLength
0,Benign,-4.876201,to,True,False,18,False,83
1,Benign,-4.920700,to,True,False,23,False,83
2,Benign,-4.821630,to,True,False,22,False,83
3,Benign,-4.889040,to,True,False,18,False,83
4,Benign,-4.772234,to,True,False,18,False,83
